<a href="https://colab.research.google.com/github/ivmarchuk/simple-sql/blob/main/Simple_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
sql_query = """
               CREATE TABLE IF NOT EXISTS FlightLeg
               (id INTEGER PRIMARY KEY AUTOINCREMENT, 
               tailNumber INTEGER,
               source_airport_code INTEGER,
               source_country_code INTEGER,
               destination_airport_code INTEGER,
               destination_country_code INTEGER,
               departure_time VARCHAR(255),
               landing_time VARCHAR(255))
    """

cursor.execute(sql_query)
print("Opened database successfully")

In [ ]:
try:
  df.to_sql("FlightLeg", conn, index=False, if_exists='append')
except:
  print("Something went wrong")
  

In [ ]:
sql_query = '''SELECT * FROM FlightLeg LIMIT 30;'''
cursor.execute(sql_query)
records = cursor.fetchall()
print("Total rows are:  ", len(records))
print("Printing each row")
for record in records:
  print(record)

In [ ]:
# SHOW DOMESTIC AND INTERNATIONAL / FLIGHT DURATION
sql_alter = '''create table flightlegs_working 
            as select 
	  ROW_NUMBER () OVER (ORDER BY tailnumber) as id,
    tailnumber,
    source_airport_code, 
    source_country_code, 
    destination_airport_code, 
    destination_country_code,
    departure_time,
    landing_time,
    ROUND((JULIANDAY(landing_time) - JULIANDAY(departure_time)) * 24 * 60, 0) as flight_duration,
    case when source_country_code = destination_country_code then 'D' else 'I' end as flight_type
	from flightlegs;''' 


cursor.execute(sql_alter)
records = cursor.fetchall()

print("Total rows are:  ", len(records))
print("Printing each row")
for record in records:
  print(record)

In [ ]:
# WHICH AIRCRAFT HAD THE MOST FLIGHTS
sql_which_aircraft_has_the_most_flights = ''' 
WITH flight_count AS (
        SELECT tailnumber, COUNT(tailnumber) AS flights
        FROM FlightLeg
        GROUP BY tailnumber)
  
SELECT tailnumber, flights
FROM flight_count
WHERE flights IN (SELECT MAX(flights) FROM flight_count); 
'''

cursor.execute(sql_which_aircraft_has_the_most_flights)
records = cursor.fetchall()

print("Printing query results: ")
for record in records:
  print(record)

In [ ]:
# WHICH AIRCRAFT HAS FLOWN LONGER THAN ANYONE #ELSE
sql_which_aircraft_has_flown_longer = ''' 
WITH summed_flights AS (
  SELECT tailnumber, SUM(flight_duration) AS total_flights_duration 
  FROM FlightLeg 
  GROUP BY tailnumber)

SELECT tailnumber, MAX(total_flights_duration) AS total
FROM summed_flights;
'''                           

cursor.execute(sql_which_aircraft_has_flown_longer)
records = cursor.fetchall()

print("Printing query results: ")
for record in records:
  print(record)             

In [ ]:
# Longest and shortest domestic/international flight with its duration
sql_longest_shortest_I_D = '''
with max_d as (
    select flight_type, max(flight_duration) as duration from flightlegs_working group by 1),
min_d as (
    select flight_type, min(flight_duration) as duration from flightlegs_working group by 1)
    
SELECT tailnumber, flight_duration, flight_type
FROM flightlegs_working
WHERE flight_duration IN (SELECT duration FROM max_d) 
					OR flight_duration IN (select duration from min_d)
ORDER BY 3,2 DESC;  ;
'''

cursor.execute(sql_longest_shortest_I_D)
records = cursor.fetchall()

print("Printing query results: ")
for record in records:
  print(record) 

In [ ]:
# The interval between which flights of the same aircraft was the shortest and how many minutes did it last? (FOR A6-EVF IN THAT CASE)

sql_shortesrt_flight_interval_for_aircraft = """
with base_t as (
    select tailnumber, departure_time, landing_time, flight_duration from flightleg_working order by 1,2 desc),
    calc as (
    select base_t.*, 
        lag(landing_time) over (partition by tailnumber order by departure_time asc) as landing_time_prev_flight,
        date_diff('minute', landing_time_prev_flight, departure_time) as time_between_flights
        from base_t
    ),
    min_time as 
        (select Min(time_between_flights) as min_time 
		from calc 
        where time_betweeN_flights > 0)
        
select * 
from calc c 
inner join min_time t 
where c.time_betweeN_flights = t.min_time
"""

cursor.execute(sql_shortesrt_flight_interval_for_aircraft)
records = cursor.fetchall()

print("Printing query results: ")
for record in records:
  print(record) 